In [1]:
import csv
import datetime
import dotenv
import glob
import json
import os
import pprint
import time

import boto3
import pandas as pd

In [2]:
dotenv.load_dotenv()
CLAUDE_MODEL_ID = os.getenv("BEDROCK_CLAUDE_MODEL_ID")
CLAUDE_CLIENT = boto3.client("bedrock-runtime", region_name="ap-northeast-1")
FUNCTION_CALLING_NAME = "hogehoge"
FUNCTION_CALLING_DESCRIPTION = """
あなたはコンタクトセンターで音声通話テキストからお客様のニーズ(VOC)を把握する業務を10年以上行っている専門家です。
与えられたテキストは過去の通話を記録したもので、CUはお客様を表し、OPはオペレータの発言になります。
"""
INPUT_DATA_PATH = "複数行1列で音声通話テキストを保存しているcsvファイルのパス"
OUTPUT_FOLDER = "出力結果のcsvファイルを保存するフォルダのパス"

In [3]:
NAME_1_IN_FUNCTION_CALLING = "predict_positive_negative_by_LLM"
PROMPT_1_IN_FUNCTION_CALLING = "会話はコンタクトセンターでの電話のやりとりです。この会話全体を通して、お客様の感情をポジティブかネガティブかを判定してください。"
NAME_2_IN_FUNCTION_CALLING = "suggest_judge_reason_by_LLM"
PROMPT_2_IN_FUNCTION_CALLING = "{a}の判定を行った理由を簡潔に出力してください。".format(a=NAME_1_IN_FUNCTION_CALLING)
NAME_3_IN_FUNCTION_CALLING = "predict_customer_emotion_score_by_LLM"
PROMPT_3_IN_FUNCTION_CALLING = "お客様の{a}の感情度を推測して数値で表してください。100を完全にポジティブ、0を完全にネガティブとします。(0-100)".format(a=NAME_1_IN_FUNCTION_CALLING)

In [4]:
def structured_output_tool():
    return {"toolSpec": {"name": FUNCTION_CALLING_NAME,
                         "description": FUNCTION_CALLING_DESCRIPTION,
                         "inputSchema": {"json": {"type": "object",
                                                  "properties": {NAME_1_IN_FUNCTION_CALLING: {"type": "string",
                                                                                              "enum": ["positive", "negative"],
                                                                                              "description": PROMPT_1_IN_FUNCTION_CALLING},
                                                                 NAME_2_IN_FUNCTION_CALLING: {"type": "string",
                                                                                              "description": PROMPT_2_IN_FUNCTION_CALLING},
                                                                 NAME_3_IN_FUNCTION_CALLING: {"type": "number",
                                                                                              "description": PROMPT_3_IN_FUNCTION_CALLING}
                                                                },
                                                  "required": [NAME_1_IN_FUNCTION_CALLING,
                                                               NAME_2_IN_FUNCTION_CALLING,
                                                               NAME_3_IN_FUNCTION_CALLING]
                                                 }
                                        }
                        }
           }


def process_claude_with_function_calling(call_text_and_speaker):
    prompt = """
    <text>
    {a}
    </text>
    """.format(a=call_text_and_speaker)
    claude_input_with_prompt = [{"role": "user",
                                 "content": [{"text": prompt}]
                                }]
    claude_response = CLAUDE_CLIENT.converse(modelId=CLAUDE_MODEL_ID,
                                             messages=claude_input_with_prompt,
                                             inferenceConfig={"temperature": 0.2},
                                             toolConfig={"tools": [structured_output_tool()],
                                                         "toolChoice": {"tool": {"name": FUNCTION_CALLING_NAME}}
                                                        })
    result = claude_response["output"]["message"]["content"][0]["toolUse"]["input"]
    return result

In [24]:
def read_csv_file_and_inference_using_claude(csv_file_path):
    call_text_and_speaker_list = []
    predict_positive_negative_list = []
    suggest_judge_reason_list = []
    predict_customer_emotion_score_list = []
    with open(file=csv_file_path, mode="r", encoding="utf-8") as csvfile:
        csvfile_data_reader = csv.reader(csvfile)
        next(csvfile_data_reader, None)  # 1行目のヘッダーをスキップする
        for row in csvfile_data_reader:
            if row:
                predict_result = process_claude_with_function_calling(call_text_and_speaker=row[0])  # 音声通話テキスト(変数row)はlistだったので[0]を設定
                call_text_and_speaker_list.append(row[0])
                predict_positive_negative_list.append(predict_result["predict_positive_negative_by_LLM"])
                suggest_judge_reason_list.append(predict_result["suggest_judge_reason_by_LLM"])
                predict_customer_emotion_score_list.append(predict_result["predict_customer_emotion_score_by_LLM"])
                time.sleep(1)
            else:
                continue
    df = pd.DataFrame(data={"comment": call_text_and_speaker_list,
                            "positive_negative": predict_positive_negative_list,
                            "judge_reason": suggest_judge_reason_list,
                            "customer_emotion_score": predict_customer_emotion_score_list})
    return df

In [ ]:
df = read_csv_file_and_inference_using_claude(csv_file_path=INPUT_DATA_PATH)
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)
df.to_csv(path_or_buf="{a}/ファイル名.csv".format(a=OUTPUT_FOLDER),
          index=False,
          encoding="utf-8")